# Examples Codes

## Running Production Applications

In [1]:
! spark-submit --class org.apache.spark.examples.SparkPi --master local "C:\spark\spark-3.5.3-bin-hadoop3\examples\jars\spark-examples_2.12-3.5.3.jar" 3

Pi is roughly 3.136890456301521


24/11/17 13:03:45 INFO SparkContext: Running Spark version 3.5.3
24/11/17 13:03:45 INFO SparkContext: OS info Windows 11, 10.0, amd64
24/11/17 13:03:45 INFO SparkContext: Java version 17.0.12
24/11/17 13:03:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/17 13:03:46 INFO ResourceUtils: ==============================================================
24/11/17 13:03:46 INFO ResourceUtils: No custom resources configured for spark.driver.
24/11/17 13:03:46 INFO ResourceUtils: ==============================================================
24/11/17 13:03:46 INFO SparkContext: Submitted application: Spark Pi
24/11/17 13:03:46 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, a

## Structured Streaming

In [44]:
from pyspark.sql import SparkSession

In [301]:
spark = SparkSession\
    .builder\
    .appName('Ex01')\
    .config("spark.executor.memory", "6g") \
    .getOrCreate()
spark

In [128]:
staticDataFrame = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("../../../data/retail-data/by-day/*.csv")
    
staticDataFrame.createOrReplaceTempView("retail_data")

staticSchema = staticDataFrame.schema

In [130]:
from pyspark.sql.functions import window, column, desc, col

staticDataFrame\
    .selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")\
    .groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
    .sum("total_cost")\
    .show()

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   14075.0|{2011-12-04 19:00...|316.78000000000003|
|   18180.0|{2011-12-04 19:00...|            310.73|
|   15358.0|{2011-12-04 19:00...| 830.0600000000003|
|   15392.0|{2011-12-04 19:00...|304.40999999999997|
|   15290.0|{2011-12-04 19:00...|263.02000000000004|
|   16811.0|{2011-12-04 19:00...|             232.3|
|   12748.0|{2011-12-04 19:00...| 363.7899999999999|
|   16500.0|{2011-12-04 19:00...| 52.74000000000001|
|   16873.0|{2011-12-04 19:00...|1854.8300000000002|
|   14060.0|{2011-12-04 19:00...|297.47999999999996|
|   14649.0|{2011-12-04 19:00...| 513.9899999999998|
|   16904.0|{2011-12-04 19:00...| 349.0200000000001|
|   17857.0|{2011-12-04 19:00...|            2979.6|
|   14083.0|{2011-12-04 19:00...| 446.5700000000001|
|   14777.0|{2011-12-04 19:00...|             -2.95|
|   16684.0|{2011-12-04 19:00...| 5401.9799999

In [131]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [142]:
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("../../../data/retail-data/by-day/*.csv")

In [143]:
streamingDataFrame.isStreaming

True

In [174]:
purchaseByCustomerPerHour = streamingDataFrame \
    .selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate") \
    .groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day")) \
    .sum("total_cost")

In [175]:
query = purchaseByCustomerPerHour.writeStream \
    .format("memory") \
    .queryName("customer_purchases") \
    .outputMode("complete") \
    .start()

In [219]:
spark.sql("""
    SELECT *
    FROM customer_purchases
    ORDER BY `sum(total_cost)` DESC
""").show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   17450.0|{2011-09-19 19:00...|          71601.44|
|      NULL|{2011-11-13 19:00...|          55316.08|
|      NULL|{2011-11-06 19:00...|          42939.17|
|      NULL|{2011-03-28 19:00...| 33521.39999999998|
|      NULL|{2011-12-07 19:00...|31975.590000000007|
+----------+--------------------+------------------+
only showing top 5 rows



In [218]:
spark.sql("""
    SELECT max(window)
    FROM customer_purchases
""").show()

+--------------------+
|         max(window)|
+--------------------+
|{2011-12-08 19:00...|
+--------------------+



In [220]:
query.stop()

## Machine Learning and Advance Analytics

In [269]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [270]:
from pyspark.sql.functions import date_format, col

preppedDataFrame = staticDataFrame.na.fill(0)\
    .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
    .coalesce(5)

In [271]:
trainDataFrame = preppedDataFrame.where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame.where("InvoiceDate >= '2011-07-01'")

In [272]:
print("Cantidad de Registros en Entrenamiento",trainDataFrame.count())
print("Cantidad de Registros en Testeo",testDataFrame.count())

Cantidad de Registros en Entrenamiento 245903
Cantidad de Registros en Testeo 296006


In [273]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer().setInputCol("day_of_week").setOutputCol("day_of_week_index")

In [274]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder().setInputCol("day_of_week_index").setOutputCol("day_of_week_encoded")

In [275]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler().setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"]).setOutputCol("features")

In [276]:
from pyspark.ml import Pipeline

transformationPipeline = Pipeline().setStages([indexer, encoder, vectorAssembler])

In [277]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

In [278]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

In [279]:
transformedTraining.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

In [280]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(20).setSeed(1)

In [281]:
kmModel = kmeans.fit(transformedTraining)

In [286]:
from pyspark.ml.evaluation import ClusteringEvaluator

silhouette_score=[] 

evaluator = ClusteringEvaluator(featuresCol='features',metricName='silhouette', distanceMeasure='squaredEuclidean') 
for i in range(2,10): 
    kmeans=KMeans(k=i) 
    model=kmeans.fit(transformedTraining) 
    predictions=model.transform(transformedTraining) 
    score=evaluator.evaluate(predictions)
    silhouette_score.append(score) 
    print('Silhouette Score for k =',i,'is',score)

Silhouette Score for k = 2 is 0.9999846815051553
Silhouette Score for k = 3 is 0.9999855493238791
Silhouette Score for k = 4 is 0.9999661006742695
Silhouette Score for k = 5 is 0.9998601800542514
Silhouette Score for k = 6 is 0.9956056825233331
Silhouette Score for k = 7 is 0.9998295365988951
Silhouette Score for k = 8 is 0.998846352119056
Silhouette Score for k = 9 is 0.9949829197816651


## Lower-level APIs

In [313]:
# No funciona posiblemente debido a que es incompatible Pyspark con Java 17, intentar conseguir Java 11.

from pyspark.sql import Row
data = [Row(value=1), Row(value=2), Row(value=3)] 
rdd = spark.sparkContext.parallelize(data) # Convert RDD to DataFrame 
df = rdd.toDF() # Show the DataFrame 
df.show()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 5) (Pino executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator